# PB testing

In [ ]:
import pandas as pd
import numpy as np

#Packages for GUIs (widgets) in python
from tkinter import Tk, filedialog
from ipywidgets import *
from IPython.display import clear_output, display

import os

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

In [ ]:
from ast import literal_eval
import time

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.chdir('D:/git/weill-neurohub/rcs-simulation')
from rcssim import rcs_sim as rcs

# Load data

In [ ]:
root = Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

print('Select folder containing data and settings files.')
data_folder = filedialog.askdirectory()

root.destroy()

# PB1

### Load measured datasets and settings

In [ ]:
# left_neural_data = pd.read_csv(left_file_name)
data_meas = pd.read_csv(data_folder + '/dataset_pb1.csv')
settings = pd.read_csv(data_folder + '/dataset_pb1_config.csv')
settings['band_edges_hz'] = settings['band_edges_hz'].apply(literal_eval)
amp_gains = np.genfromtxt(data_folder + '/amp_gains.csv', 
                          delimiter=',').astype(int)

In [ ]:
print('Settings')
settings

### TD --> PB

In [ ]:
data_sim = data_meas[['timestamp', 'td1']].copy()
data_sim['pb1'] = np.nan*np.ones(np.shape(data_sim['td1']))

t_start = time.time()
hann_win = rcs.create_hann_window(settings.loc[0,'fft_size'], percent=100)

# Compute power band for the first time-domain channel
data_td = rcs.transform_mv_to_rcs(data_sim['td1'].values, amp_gains[0])
data_fft, t_pb = rcs.td_to_fft(data_td, data_sim['timestamp'].values, 
                               fs_td=settings.fs_td[0], 
                               L=settings.fft_size[0], 
                               interval=settings.interval[0], 
                               hann_win=hann_win, output_in_mv=False)
data_pb = rcs.fft_to_pb(data_fft, fs_td=500, L=256, 
                        bit_shift=settings.bit_shift[0], 
                        band_edges_hz=settings.band_edges_hz[0], 
                        input_is_mv=False)
pb_sample_mask = np.isin(data_sim.timestamp, t_pb)
data_sim.loc[pb_sample_mask,'pb1'] = data_pb

t_end = time.time()
print('Time elapsed: ' + str(t_end-t_start))

In [ ]:
# %matplotlib widget
fig, ax = plt.subplots(1,1, figsize=(6,3), sharex='col', sharey=False)

ax.plot(data_meas.loc[~np.isnan(data_meas.pb1), 'timestamp'], 
           data_meas.loc[~np.isnan(data_meas.pb1), 'pb1'], label='Measured')
ax.plot(data_sim.loc[~np.isnan(data_sim.pb1), 'timestamp'], 
           data_sim.loc[~np.isnan(data_sim.pb1), 'pb1'], label='Simulated')
ax.legend()

ax.grid()
ax.set_xlabel('Time [sec]')
ax.set_ylabel('PB1 Output \n [RCS units]')

plt.tight_layout()

# PB2

### Load measured datasets and settings

In [ ]:
# left_neural_data = pd.read_csv(left_file_name)
data_meas = pd.read_csv(data_folder + '/dataset_pb2.csv')
settings = pd.read_csv(data_folder + '/dataset_pb2_config.csv')
settings['band_edges_hz'] = settings['band_edges_hz'].apply(literal_eval)
amp_gains = np.genfromtxt(data_folder + '/amp_gains.csv', 
                          delimiter=',').astype(int)

In [ ]:
print('Settings')
settings

### TD --> PB

In [ ]:
data_sim = data_meas[['timestamp', 'td1', 'td2']].copy()

t_start = time.time()
hann_win = rcs.create_hann_window(settings.fft_size[0], percent=100)

# Transform the units for each time-domain channel
data_td1 = rcs.transform_mv_to_rcs(data_sim['td1'].values, amp_gains[0])
data_td2 = rcs.transform_mv_to_rcs(data_sim['td2'].values, amp_gains[1])

# Compute all power bands for their respective time-domain channels
for i, pb in enumerate(settings.loc[0, 'band_edges_hz']):
    pb_lbl = data_meas.keys()[i+3]
    data_sim[pb_lbl] = np.nan*np.ones(np.shape(data_sim['td1']))
    start_ts = np.min(data_meas.loc[~np.isnan(data_meas[pb_lbl]), 'timestamp'])
    end_ts = np.max(data_meas.loc[~np.isnan(data_meas[pb_lbl]), 'timestamp'])
    
    if pb[0]==1:        
        data_fft, t_pb = rcs.td_to_fft(data_td1, 
                                       data_sim['timestamp'].values, 
                                       fs_td=settings.fs_td[0], 
                                       L=settings.fft_size[0], 
                                       interval=settings.interval[0], 
                                       hann_win=hann_win)
    elif pb[0]==2:
        data_fft, t_pb = rcs.td_to_fft(data_td2, 
                                       data_sim['timestamp'].values, 
                                       fs_td=settings.fs_td[0], 
                                       L=settings.fft_size[0], 
                                       interval=settings.interval[0], 
                                       hann_win=hann_win)
        
    data_pb = rcs.fft_to_pb(data_fft, settings.fs_td[0], 
                            L = settings.fft_size[0], 
                            bit_shift=settings.bit_shift[0], 
                            band_edges_hz=[pb[1]], 
                            input_is_mv=False)
    data_pb[(t_pb < start_ts+5) | (t_pb > end_ts)] = np.nan
    pb_sample_mask = np.isin(data_sim.timestamp, t_pb)
    data_sim.loc[pb_sample_mask, pb_lbl] = data_pb


t_end = time.time()
print('Time elapsed: ' + str(t_end-t_start))

In [ ]:
# %matplotlib widget

fig, ax = plt.subplots(19,1, figsize=(6,38))
cols = data_meas.keys()[-19:]

for i in np.arange(19):
    ax[i].plot(data_meas.loc[~np.isnan(data_meas[cols[i]]), 'timestamp'], 
               data_meas.loc[~np.isnan(data_meas[cols[i]]), cols[i]], label='Measured')
    ax[i].plot(data_sim.loc[~np.isnan(data_sim[cols[i]]), 'timestamp'], 
               data_sim.loc[~np.isnan(data_sim[cols[i]]), cols[i]], label='Simulated')
    ax[i].legend()

    ax[i].grid()
    ax[i].set_ylim([0, 1.2*np.max(data_sim.loc[~np.isnan(data_sim[cols[i]]), 
                                               cols[i]])])
    ax[i].set_title(f'{cols[i][2:]}Hz Input')
    ax[i].set_ylabel('PB output  \n [RCS units]')
    
ax[18].set_xlabel('Time [sec]')

plt.tight_layout()

# PB3a

### Load measured datasets and settings

In [ ]:
# left_neural_data = pd.read_csv(left_file_name)
data_meas = pd.read_csv(data_folder + '/dataset_pb3a.csv')
settings = pd.read_csv(data_folder + '/dataset_pb3a_config.csv')
settings['band_edges_hz'] = settings['band_edges_hz'].apply(literal_eval)
amp_gains = np.genfromtxt(data_folder + '/amp_gains.csv', 
                          delimiter=',').astype(int)

In [ ]:
print('Settings')
settings

### TD --> PB

In [ ]:
data_sim = data_meas[['timestamp', 'td1']].copy()
data_sim['pb1'] = np.nan*np.ones(np.shape(data_sim['td1']))

t_start = time.time()
hann_win = rcs.create_hann_window(settings.fft_size[0], percent=100)

# Compute power band for the first time-domain channel
data_td = rcs.transform_mv_to_rcs(data_sim['td1'].values, amp_gains[0])
time_td = data_sim['timestamp'].values[~np.isnan(data_td)]
data_td = data_td[~np.isnan(data_td)]
data_fft, t_pb = rcs.td_to_fft(data_td, time_td, 
                               fs_td=settings.fs_td[0], 
                               L=settings.fft_size[0], 
                               interval=settings.interval[0], 
                               hann_win=hann_win)
data_pb = rcs.fft_to_pb(data_fft, 
                        fs_td=settings.fs_td[0], 
                        L=settings.fft_size[0], 
                        bit_shift=settings.bit_shift[0], 
                        band_edges_hz=[settings.band_edges_hz[0]])
pb_sample_mask = np.isin(data_sim.timestamp, t_pb)
data_sim.loc[pb_sample_mask,'pb1'] = data_pb

t_end = time.time()
print('Time elapsed: ' + str(t_end-t_start))

In [ ]:
# %matplotlib widget
fig, ax = plt.subplots(1,1, figsize=(6,3), sharex='col', sharey=False)

ax.plot(data_meas.loc[~np.isnan(data_meas.pb1), 'timestamp'], 
           data_meas.loc[~np.isnan(data_meas.pb1), 'pb1'], label='Measured')
ax.plot(data_sim.loc[~np.isnan(data_sim.pb1), 'timestamp'], 
           data_sim.loc[~np.isnan(data_sim.pb1), 'pb1'], label='Simulated')
ax.legend(bbox_to_anchor=(1.02, 0.6))\

ax.grid()
ax.set_xlabel('Time [sec]')
ax.set_ylabel('PB1 Output \n [RCS units]')
ax.set_title('$f_{s,TD}=250Hz, L=64pt$')

plt.tight_layout()

# PB3b

### Load measured datasets and settings

In [ ]:
# left_neural_data = pd.read_csv(left_file_name)
data_meas = pd.read_csv(data_folder + '/dataset_pb3b.csv')
settings = pd.read_csv(data_folder + '/dataset_pb3b_config.csv')
settings['band_edges_hz'] = settings['band_edges_hz'].apply(literal_eval)
amp_gains = np.genfromtxt(data_folder + '/amp_gains.csv', 
                          delimiter=',').astype(int)

In [ ]:
print('Settings')
settings

### TD --> PB

In [ ]:
data_sim = data_meas[['timestamp', 'td1']].copy()
data_sim['pb1'] = np.nan*np.ones(np.shape(data_sim['td1']))

t_start = time.time()
hann_win = rcs.create_hann_window(settings.fft_size[0], percent=100)

# Compute power band for the first time-domain channel
data_td = rcs.transform_mv_to_rcs(data_sim['td1'].values, amp_gains[0])
time_td = data_sim['timestamp'].values[~np.isnan(data_td)]
data_td = data_td[~np.isnan(data_td)]
data_fft, t_pb = rcs.td_to_fft(data_td, time_td, 
                               fs_td=settings.fs_td[0], 
                               L=settings.fft_size[0], 
                               interval=settings.interval[0], 
                               hann_win=hann_win)
data_pb = rcs.fft_to_pb(data_fft, 
                        fs_td=settings.fs_td[0], 
                        L=settings.fft_size[0], 
                        bit_shift=settings.bit_shift[0], 
                        band_edges_hz=[settings.band_edges_hz[0]])
pb_sample_mask = np.isin(data_sim.timestamp, t_pb)
data_sim.loc[pb_sample_mask,'pb1'] = data_pb

t_end = time.time()
print('Time elapsed: ' + str(t_end-t_start))

In [ ]:
# %matplotlib widget
fig, ax = plt.subplots(1,1, figsize=(6,3), sharex='col', sharey=False)

ax.plot(data_meas.loc[~np.isnan(data_meas.pb1), 'timestamp'], 
           data_meas.loc[~np.isnan(data_meas.pb1), 'pb1'], label='Measured')
ax.plot(data_sim.loc[~np.isnan(data_sim.pb1), 'timestamp'], 
           data_sim.loc[~np.isnan(data_sim.pb1), 'pb1'], label='Simulated')
ax.legend(bbox_to_anchor=(1.02, 0.6))\

ax.grid()
ax.set_xlabel('Time [sec]')
ax.set_ylabel('PB1 Output \n [RCS units]')
ax.set_title('$f_{s,TD}=250Hz, L=256pt$')

plt.tight_layout()

# PB3c

### Load measured datasets and settings

In [ ]:
# left_neural_data = pd.read_csv(left_file_name)
data_meas = pd.read_csv(data_folder + '/dataset_pb3c.csv')
settings = pd.read_csv(data_folder + '/dataset_pb3c_config.csv')
settings['band_edges_hz'] = settings['band_edges_hz'].apply(literal_eval)
amp_gains = np.genfromtxt(data_folder + '/amp_gains.csv', 
                          delimiter=',').astype(int)

In [ ]:
print('Settings')
settings

### TD --> PB

In [ ]:
data_sim = data_meas[['timestamp', 'td1']].copy()
data_sim['pb1'] = np.nan*np.ones(np.shape(data_sim['td1']))

t_start = time.time()
hann_win = rcs.create_hann_window(settings.fft_size[0], percent=100)

# Compute power band for the first time-domain channel
data_td = rcs.transform_mv_to_rcs(data_sim['td1'].values, amp_gains[0])
time_td = data_sim['timestamp'].values[~np.isnan(data_td)]
data_td = data_td[~np.isnan(data_td)]
data_fft, t_pb = rcs.td_to_fft(data_td, time_td, 
                               fs_td=settings.fs_td[0], 
                               L=settings.fft_size[0], 
                               interval=settings.interval[0], 
                               hann_win=hann_win)
data_pb = rcs.fft_to_pb(data_fft, 
                        fs_td=settings.fs_td[0], 
                        L=settings.fft_size[0], 
                        bit_shift=settings.bit_shift[0], 
                        band_edges_hz=[settings.band_edges_hz[0]])
pb_sample_mask = np.isin(data_sim.timestamp, t_pb)
data_sim.loc[pb_sample_mask,'pb1'] = data_pb

t_end = time.time()
print('Time elapsed: ' + str(t_end-t_start))

In [ ]:
# %matplotlib widget
fig, ax = plt.subplots(1,1, figsize=(6,3), sharex='col', sharey=False)

ax.plot(data_meas.loc[~np.isnan(data_meas.pb1), 'timestamp'], 
           data_meas.loc[~np.isnan(data_meas.pb1), 'pb1'], label='Measured')
ax.plot(data_sim.loc[~np.isnan(data_sim.pb1), 'timestamp'], 
           data_sim.loc[~np.isnan(data_sim.pb1), 'pb1'], label='Simulated')
ax.legend(bbox_to_anchor=(1.02, 0.6))\

ax.grid()
ax.set_xlabel('Time [sec]')
ax.set_ylabel('PB1 Output \n [RCS units]')
ax.set_title('$f_{s,TD}=250Hz, L=1024pt$')

plt.tight_layout()

# PB4

### Load measured datasets and settings

In [ ]:
# left_neural_data = pd.read_csv(left_file_name)
data_meas = pd.read_csv(data_folder + '/dataset_pb4.csv')
settings = pd.read_csv(data_folder + '/dataset_pb4_config.csv')
settings['bit_shift'] = settings['bit_shift'].apply(literal_eval)
settings['band_edges_hz'] = settings['band_edges_hz'].apply(literal_eval)
amp_gains = np.genfromtxt(data_folder + '/amp_gains.csv', 
                          delimiter=',').astype(int)

In [ ]:
print('Settings')
settings

### TD --> PB

In [ ]:
data_sim = data_meas[['timestamp', 'td1']].copy()
data_sim['pb1'] = np.nan*np.ones(np.shape(data_sim['td1']))

t_start = time.time()
hann_win = rcs.create_hann_window(settings.fft_size[0], percent=100)

# Compute power band for the first time-domain channel
data_td = rcs.transform_mv_to_rcs(data_sim['td1'].values, amp_gains[0])
data_fft, t_pb = rcs.td_to_fft(data_td, data_sim['timestamp'].values, 
                               fs_td=settings.fs_td[0], 
                               L=settings.fft_size[0], 
                               interval=settings.interval[0], 
                               hann_win=hann_win)
for i, bit_shift in enumerate(settings.loc[0, 'bit_shift']):
    pb_lbl = data_meas.keys()[i+2]
    data_sim[pb_lbl] = np.nan*np.ones(np.shape(data_sim['td1']))
    start_ts = np.min(data_meas.loc[~np.isnan(data_meas[pb_lbl]), 'timestamp'])
    end_ts = np.max(data_meas.loc[~np.isnan(data_meas[pb_lbl]), 'timestamp'])
    
    data_pb = rcs.fft_to_pb(data_fft, 
                            fs_td=settings.fs_td[0], 
                            L=settings.fft_size[0], 
                            bit_shift=bit_shift, 
                            band_edges_hz=[settings.band_edges_hz[0]])
    data_pb[(t_pb < start_ts+2) | (t_pb > end_ts)] = np.nan
    pb_sample_mask = np.isin(data_sim.timestamp, t_pb)
    data_sim.loc[pb_sample_mask, pb_lbl] = data_pb

t_end = time.time()
print('Time elapsed: ' + str(t_end-t_start))

In [ ]:
# %matplotlib widget
fig, ax = plt.subplots(1,1, figsize=(8,3), sharex='col', sharey=False)

ax.plot(data_meas.loc[~np.isnan(data_meas['pb1']), 'timestamp'], 
        data_meas.loc[~np.isnan(data_meas['pb1']), 'pb1'], 
        color='black', label=r'$\bf{Measured}$')
ax.plot([0],[0], color='white', label=' ')
ax.plot([0],[0], color='white', label=r'$\bf{Simulated}$')
for i in range(8):
    pb_lbl = data_meas.keys()[i+2]
    ax.plot(data_meas.loc[~np.isnan(data_meas[pb_lbl]), 'timestamp'], 
            data_meas.loc[~np.isnan(data_meas[pb_lbl]), pb_lbl], 
            color='black')
    ax.plot(data_sim.loc[~np.isnan(data_sim[pb_lbl]), 'timestamp'], 
            data_sim.loc[~np.isnan(data_sim[pb_lbl]), pb_lbl], 
            label=f'Bit shift={i}')

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid()
ax.set_xlabel('Time [sec]')
ax.set_ylabel('PB1 Output \n [RCS units]')
ax.set_xlim([0,380])

plt.tight_layout()